# Modelos de Ecuaciones Estructurales (SEM) - Por Tipo de Ítem

Este notebook implementa **modelos de path analysis** para analizar cómo la ideología predice cambios de opinión/tiempo **según el tipo de ítem** (Progresistas vs Conservadores).

## Objetivo:

Determinar si la ideología del sujeto predice **diferente** los cambios en ítems progresistas vs conservadores.

## Variables Creadas:

Agregamos cambios por tipo de ítem (sumando todas las direcciones):

**Cambio de Opinión (CO):**
1. `CO_Total_Progresistas` = Cambio_Op_Sum_Pro_Izq + Cambio_Op_Sum_Pro_Der
2. `CO_Total_Conservadores` = Cambio_Op_Sum_Con_Izq + Cambio_Op_Sum_Con_Der

**Cambio de Tiempo (CT):**
3. `CT_Total_Progresistas` = Cambio_Tiempo_Sum_Pro_Izq + Cambio_Tiempo_Sum_Pro_Der
4. `CT_Total_Conservadores` = Cambio_Tiempo_Sum_Con_Izq + Cambio_Tiempo_Sum_Con_Der

## Modelo para cada variable:

```
Indice_Progresismo ────→ Y
                          ↑
Indice_Conservadurismo ──→
```

## Hipótesis a Probar:

**H1:** Mayor progresismo predice **más cambios** en ítems progresistas  
**H2:** Mayor conservadurismo predice **más cambios** en ítems conservadores  
**H3:** La ideología explica **mejor** los cambios en ítems congruentes con ella  
**H4:** Los coeficientes β difieren entre ítems progresistas vs conservadores

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

from statsmodels.api import OLS, add_constant
from sklearn.preprocessing import StandardScaler

from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

print("✓ Librerías cargadas exitosamente")

## 1. Cargar Datos

In [ ]:
# Rutas
Ruta_Base = os.path.join(os.getcwd(), '..', 'Data', 'Bases definitivas')
Excel_Generales = os.path.join(Ruta_Base, 'Generales.xlsx')
Excel_Ballotage = os.path.join(Ruta_Base, 'Ballotage.xlsx')

# Cargar DataFrames
df_Generales = pd.read_excel(Excel_Generales)
df_Ballotage = pd.read_excel(Excel_Ballotage)

dfs_Finales = {
    'Generales': df_Generales,
    'Ballotage': df_Ballotage
}

print(f"✓ Datos cargados:")
print(f"  - Generales: {len(df_Generales)} registros")
print(f"  - Ballotage: {len(df_Ballotage)} registros")

## 2. Crear Variables Agregadas por Tipo de Ítem

In [ ]:
print("="*70)
print("CREANDO VARIABLES AGREGADAS POR TIPO DE ÍTEM")
print("="*70)

# Variables base necesarias
Vars_CO_Pro = ['Cambio_Op_Sum_Pro_Izq', 'Cambio_Op_Sum_Pro_Der']
Vars_CO_Con = ['Cambio_Op_Sum_Con_Izq', 'Cambio_Op_Sum_Con_Der']
Vars_CT_Pro = ['Cambio_Tiempo_Sum_Pro_Izq', 'Cambio_Tiempo_Sum_Pro_Der']
Vars_CT_Con = ['Cambio_Tiempo_Sum_Con_Izq', 'Cambio_Tiempo_Sum_Con_Der']

for nombre_df, df in dfs_Finales.items():
    print(f"\n📊 {nombre_df}:")
    print("-"*70)
    
    # CO Total Progresistas
    if all(v in df.columns for v in Vars_CO_Pro):
        df['CO_Total_Progresistas'] = df[Vars_CO_Pro].sum(axis=1)
        media = df['CO_Total_Progresistas'].mean()
        print(f"  ✓ CO_Total_Progresistas creada (media = {media:.2f})")
    else:
        print(f"  ❌ Variables CO_Pro faltantes")
    
    # CO Total Conservadores
    if all(v in df.columns for v in Vars_CO_Con):
        df['CO_Total_Conservadores'] = df[Vars_CO_Con].sum(axis=1)
        media = df['CO_Total_Conservadores'].mean()
        print(f"  ✓ CO_Total_Conservadores creada (media = {media:.2f})")
    else:
        print(f"  ❌ Variables CO_Con faltantes")
    
    # CT Total Progresistas
    if all(v in df.columns for v in Vars_CT_Pro):
        df['CT_Total_Progresistas'] = df[Vars_CT_Pro].sum(axis=1)
        media = df['CT_Total_Progresistas'].mean()
        print(f"  ✓ CT_Total_Progresistas creada (media = {media:.2f})")
    else:
        print(f"  ❌ Variables CT_Pro faltantes")
    
    # CT Total Conservadores
    if all(v in df.columns for v in Vars_CT_Con):
        df['CT_Total_Conservadores'] = df[Vars_CT_Con].sum(axis=1)
        media = df['CT_Total_Conservadores'].mean()
        print(f"  ✓ CT_Total_Conservadores creada (media = {media:.2f})")
    else:
        print(f"  ❌ Variables CT_Con faltantes")

print("\n" + "="*70)
print("✅ VARIABLES AGREGADAS CREADAS")
print("="*70)

## 3. Definir Variables para Modelos

In [ ]:
# Variables predictoras
Predictores = [
    'Indice_Progresismo',
    'Indice_Conservadurismo'
]

# Variables dependientes - Por Tipo de Ítem
Outcomes_Tipo_Item = [
    'CO_Total_Progresistas',
    'CO_Total_Conservadores',
    'CT_Total_Progresistas',
    'CT_Total_Conservadores'
]

print(f"✓ Variables definidas:")
print(f"  - Predictores: {len(Predictores)}")
print(f"  - Outcomes: {len(Outcomes_Tipo_Item)}")

## 4. Función para Ejecutar Modelo SEM

In [ ]:
def Ejecutar_Modelo_Path(df, outcome, predictores, nombre_modelo):
    """
    Ejecuta un modelo de path analysis (regresión múltiple).
    """
    
    # Verificar variables
    variables_necesarias = [outcome] + predictores
    variables_existentes = [v for v in variables_necesarias if v in df.columns]
    
    if len(variables_existentes) != len(variables_necesarias):
        faltantes = set(variables_necesarias) - set(variables_existentes)
        print(f"  ⚠️  Variables faltantes: {faltantes}")
        return None
    
    # Seleccionar datos
    df_modelo = df[variables_necesarias].dropna()
    
    if len(df_modelo) < 10:
        print(f"  ⚠️  Datos insuficientes: n={len(df_modelo)}")
        return None
    
    try:
        # Preparar datos
        X = df_modelo[predictores]
        y = df_modelo[outcome]
        
        # Modelo con constante
        X_const = add_constant(X)
        modelo_ols = OLS(y, X_const).fit()
        
        # Coeficientes estandarizados
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        
        X_std = scaler_X.fit_transform(X)
        y_std = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()
        
        modelo_std = OLS(y_std, X_std).fit()
        
        # Resultados
        resultados = {
            'Modelo': nombre_modelo,
            'Outcome': outcome,
            'n': len(df_modelo),
            'R²': modelo_ols.rsquared,
            'R²_ajustado': modelo_ols.rsquared_adj,
            'AIC': modelo_ols.aic,
            'BIC': modelo_ols.bic,
            'F_stat': modelo_ols.fvalue,
            'F_pvalue': modelo_ols.f_pvalue,
            'Coeficientes': {}
        }
        
        # Coeficientes
        for i, pred in enumerate(predictores):
            resultados['Coeficientes'][pred] = {
                'β': modelo_ols.params[pred],
                'β_std': modelo_std.params[i],
                'SE': modelo_ols.bse[pred],
                't': modelo_ols.tvalues[pred],
                'p': modelo_ols.pvalues[pred]
            }
        
        return resultados
        
    except Exception as e:
        print(f"  ❌ Error: {e}")
        return None

## 5. Ejecutar Modelos para GENERALES

In [ ]:
print("="*70)
print("EJECUTANDO MODELOS SEM: GENERALES (Por Tipo de Ítem)")
print("="*70)

resultados_generales = []

for outcome in Outcomes_Tipo_Item:
    print(f"\n📊 Modelo: {outcome}")
    print("-"*70)
    
    resultado = Ejecutar_Modelo_Path(
        df_Generales,
        outcome,
        Predictores,
        f"Generales_{outcome}"
    )
    
    if resultado:
        resultados_generales.append(resultado)
        
        print(f"  n = {resultado['n']}")
        print(f"  R² = {resultado['R²']:.4f}")
        print(f"  R²_ajustado = {resultado['R²_ajustado']:.4f}")
        print(f"  F({len(Predictores)}, {resultado['n']-len(Predictores)-1}) = {resultado['F_stat']:.2f}, p = {resultado['F_pvalue']:.4f}")
        print(f"\n  Coeficientes:")
        
        for pred, coefs in resultado['Coeficientes'].items():
            beta = coefs['β']
            beta_std = coefs['β_std']
            p_val = coefs['p']
            sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else 'ns'
            
            print(f"    {pred:<25} β = {beta:>7.4f}  β_std = {beta_std:>6.3f}  (p = {p_val:.4f}) {sig}")
        
        if resultado['F_pvalue'] < 0.05:
            print(f"\n  ✅ Modelo significativo")
        else:
            print(f"\n  ⚠️  Modelo NO significativo")

print(f"\n{'-'*70}")
print(f"✅ {len(resultados_generales)} modelos ejecutados para Generales")
print("="*70)

## 6. Ejecutar Modelos para BALLOTAGE

In [ ]:
print("="*70)
print("EJECUTANDO MODELOS SEM: BALLOTAGE (Por Tipo de Ítem)")
print("="*70)

resultados_ballotage = []

for outcome in Outcomes_Tipo_Item:
    print(f"\n📊 Modelo: {outcome}")
    print("-"*70)
    
    resultado = Ejecutar_Modelo_Path(
        df_Ballotage,
        outcome,
        Predictores,
        f"Ballotage_{outcome}"
    )
    
    if resultado:
        resultados_ballotage.append(resultado)
        
        print(f"  n = {resultado['n']}")
        print(f"  R² = {resultado['R²']:.4f}")
        print(f"  R²_ajustado = {resultado['R²_ajustado']:.4f}")
        print(f"  F({len(Predictores)}, {resultado['n']-len(Predictores)-1}) = {resultado['F_stat']:.2f}, p = {resultado['F_pvalue']:.4f}")
        print(f"\n  Coeficientes:")
        
        for pred, coefs in resultado['Coeficientes'].items():
            beta = coefs['β']
            beta_std = coefs['β_std']
            p_val = coefs['p']
            sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else 'ns'
            
            print(f"    {pred:<25} β = {beta:>7.4f}  β_std = {beta_std:>6.3f}  (p = {p_val:.4f}) {sig}")
        
        if resultado['F_pvalue'] < 0.05:
            print(f"\n  ✅ Modelo significativo")
        else:
            print(f"\n  ⚠️  Modelo NO significativo")

print(f"\n{'-'*70}")
print(f"✅ {len(resultados_ballotage)} modelos ejecutados para Ballotage")
print("="*70)

## 7. Crear Tablas de Resultados

In [ ]:
def Crear_Tabla_Resultados(lista_resultados):
    """Convierte resultados en DataFrames."""
    
    # Métricas
    metricas_data = []
    for res in lista_resultados:
        metricas_data.append({
            'Outcome': res['Outcome'],
            'n': res['n'],
            'R²': res['R²'],
            'R²_ajustado': res['R²_ajustado'],
            'AIC': res['AIC'],
            'BIC': res['BIC'],
            'F_stat': res['F_stat'],
            'F_pvalue': res['F_pvalue']
        })
    
    df_metricas = pd.DataFrame(metricas_data)
    
    # Coeficientes
    coef_data = []
    for res in lista_resultados:
        for pred, coefs in res['Coeficientes'].items():
            p_val = coefs['p']
            sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else 'ns'
            
            coef_data.append({
                'Outcome': res['Outcome'],
                'Predictor': pred,
                'β': coefs['β'],
                'β_std': coefs['β_std'],
                'SE': coefs['SE'],
                't': coefs['t'],
                'p-valor': p_val,
                'Sig': sig
            })
    
    df_coeficientes = pd.DataFrame(coef_data)
    
    return df_metricas, df_coeficientes

In [ ]:
# Crear tablas
df_metricas_gen, df_coef_gen = Crear_Tabla_Resultados(resultados_generales)
df_metricas_bal, df_coef_bal = Crear_Tabla_Resultados(resultados_ballotage)

print("\n📋 MÉTRICAS - GENERALES:")
print(df_metricas_gen.to_string(index=False))

print("\n📋 COEFICIENTES - GENERALES:")
print(df_coef_gen.to_string(index=False))

print("\n📋 MÉTRICAS - BALLOTAGE:")
print(df_metricas_bal.to_string(index=False))

print("\n📋 COEFICIENTES - BALLOTAGE:")
print(df_coef_bal.to_string(index=False))

## 8. Guardar Resultados en Excel

In [ ]:
# Crear carpeta
Carpeta_Resultados = os.path.join(os.getcwd(), '..', 'Data', 'Resultados_SEM')
if not os.path.exists(Carpeta_Resultados):
    os.makedirs(Carpeta_Resultados)

# Guardar Generales
archivo_gen = os.path.join(Carpeta_Resultados, 'SEM_Por_Tipo_Item_Generales.xlsx')
with pd.ExcelWriter(archivo_gen, engine='openpyxl') as writer:
    df_metricas_gen.to_excel(writer, sheet_name='Métricas de Ajuste', index=False)
    df_coef_gen.to_excel(writer, sheet_name='Coeficientes', index=False)

print(f"✓ SEM_Por_Tipo_Item_Generales.xlsx")

# Guardar Ballotage
archivo_bal = os.path.join(Carpeta_Resultados, 'SEM_Por_Tipo_Item_Ballotage.xlsx')
with pd.ExcelWriter(archivo_bal, engine='openpyxl') as writer:
    df_metricas_bal.to_excel(writer, sheet_name='Métricas de Ajuste', index=False)
    df_coef_bal.to_excel(writer, sheet_name='Coeficientes', index=False)

print(f"✓ SEM_Por_Tipo_Item_Ballotage.xlsx")

## 9. Comparación: Progresistas vs Conservadores

In [ ]:
print("="*70)
print("COMPARACIÓN: ÍTEMS PROGRESISTAS vs CONSERVADORES")
print("="*70)

print("\n📊 GENERALES - Comparación de R²:")
print("-"*70)

for tipo in ['CO', 'CT']:
    prog = df_metricas_gen[df_metricas_gen['Outcome'] == f'{tipo}_Total_Progresistas']
    cons = df_metricas_gen[df_metricas_gen['Outcome'] == f'{tipo}_Total_Conservadores']
    
    if len(prog) > 0 and len(cons) > 0:
        r2_prog = prog['R²'].values[0]
        r2_cons = cons['R²'].values[0]
        
        print(f"\n  {tipo}:")
        print(f"    Progresistas:    R² = {r2_prog:.4f}")
        print(f"    Conservadores:   R² = {r2_cons:.4f}")
        print(f"    Diferencia:      {r2_prog - r2_cons:+.4f}")
        
        if abs(r2_prog - r2_cons) < 0.01:
            print(f"    ✅ Similar poder explicativo")
        elif r2_prog > r2_cons:
            print(f"    📈 Progresistas mejor explicados")
        else:
            print(f"    📉 Conservadores mejor explicados")

print("\n📊 BALLOTAGE - Comparación de R²:")
print("-"*70)

for tipo in ['CO', 'CT']:
    prog = df_metricas_bal[df_metricas_bal['Outcome'] == f'{tipo}_Total_Progresistas']
    cons = df_metricas_bal[df_metricas_bal['Outcome'] == f'{tipo}_Total_Conservadores']
    
    if len(prog) > 0 and len(cons) > 0:
        r2_prog = prog['R²'].values[0]
        r2_cons = cons['R²'].values[0]
        
        print(f"\n  {tipo}:")
        print(f"    Progresistas:    R² = {r2_prog:.4f}")
        print(f"    Conservadores:   R² = {r2_cons:.4f}")
        print(f"    Diferencia:      {r2_prog - r2_cons:+.4f}")
        
        if abs(r2_prog - r2_cons) < 0.01:
            print(f"    ✅ Similar poder explicativo")
        elif r2_prog > r2_cons:
            print(f"    📈 Progresistas mejor explicados")
        else:
            print(f"    📉 Conservadores mejor explicados")

print("\n" + "="*70)

## 10. Análisis de Coeficientes por Tipo de Ítem

In [ ]:
print("="*70)
print("ANÁLISIS DE COEFICIENTES: ¿Qué índice predice mejor cada tipo?")
print("="*70)

print("\n📊 GENERALES:")
print("-"*70)

# Progresismo en ítems progresistas
prog_prog = df_coef_gen[
    (df_coef_gen['Outcome'].str.contains('Progresistas')) & 
    (df_coef_gen['Predictor'] == 'Indice_Progresismo')
]

# Conservadurismo en ítems conservadores
cons_cons = df_coef_gen[
    (df_coef_gen['Outcome'].str.contains('Conservadores')) & 
    (df_coef_gen['Predictor'] == 'Indice_Conservadurismo')
]

print("\n  Progresismo → Ítems Progresistas:")
for _, row in prog_prog.iterrows():
    print(f"    {row['Outcome']:<30} β_std = {row['β_std']:>6.3f} ({row['Sig']})")

print("\n  Conservadurismo → Ítems Conservadores:")
for _, row in cons_cons.iterrows():
    print(f"    {row['Outcome']:<30} β_std = {row['β_std']:>6.3f} ({row['Sig']})")

print("\n📊 BALLOTAGE:")
print("-"*70)

prog_prog_bal = df_coef_bal[
    (df_coef_bal['Outcome'].str.contains('Progresistas')) & 
    (df_coef_bal['Predictor'] == 'Indice_Progresismo')
]

cons_cons_bal = df_coef_bal[
    (df_coef_bal['Outcome'].str.contains('Conservadores')) & 
    (df_coef_bal['Predictor'] == 'Indice_Conservadurismo')
]

print("\n  Progresismo → Ítems Progresistas:")
for _, row in prog_prog_bal.iterrows():
    print(f"    {row['Outcome']:<30} β_std = {row['β_std']:>6.3f} ({row['Sig']})")

print("\n  Conservadurismo → Ítems Conservadores:")
for _, row in cons_cons_bal.iterrows():
    print(f"    {row['Outcome']:<30} β_std = {row['β_std']:>6.3f} ({row['Sig']})")

print("\n" + "="*70)

## 11. Resumen Final

In [ ]:
print("="*70)
print("RESUMEN FINAL: MODELOS SEM - POR TIPO DE ÍTEM")
print("="*70)

print("\n📊 Modelos ejecutados:")
print(f"  - Generales: {len(resultados_generales)} modelos")
print(f"  - Ballotage: {len(resultados_ballotage)} modelos")
print(f"  - Total: {len(resultados_generales) + len(resultados_ballotage)} modelos")

print("\n📈 Estadísticas generales:")
print(f"  Generales - R² promedio: {df_metricas_gen['R²'].mean():.4f}")
print(f"  Ballotage - R² promedio: {df_metricas_bal['R²'].mean():.4f}")

print("\n📁 Archivos generados:")
print("  - SEM_Por_Tipo_Item_Generales.xlsx")
print("  - SEM_Por_Tipo_Item_Ballotage.xlsx")

print("\n🎯 Preguntas respondidas:")
print("  1. ¿La ideología predice cambios en ítems progresistas?")
print("  2. ¿La ideología predice cambios en ítems conservadores?")
print("  3. ¿Hay diferencia en poder predictivo según tipo de ítem?")
print("  4. ¿El Indice_Progresismo predice mejor ítems progresistas?")
print("  5. ¿El Indice_Conservadurismo predice mejor ítems conservadores?")

print("\n💡 Interpretación:")
print("  - Si R² similar: La ideología predice ambos tipos por igual")
print("  - Si R² mayor en Progresistas: Ideología predice mejor cambios progresistas")
print("  - Si R² mayor en Conservadores: Ideología predice mejor cambios conservadores")
print("  - β_std grandes indican predictores importantes")

print("\n" + "="*70)
print("✅ ANÁLISIS SEM COMPLETADO")
print("="*70)